In [12]:
import random
import numpy as np
from numpy.random import *
import matplotlib.pyplot as plt
import scipy.stats
from scipy.special import expit
import pandas as pd
from sklearn.linear_model import LogisticRegression, LinearRegression


np.random.seed(1234)

In [2]:
import random

random.seed(1234)

In [3]:
#データ数
num_data = 200

#年齢
x_1 = randint(15, 76, num_data)

#性別(0 を女性、　１を男性)
x_2 = randint(0, 2, num_data)

# ノイズの生成
e_z = randn(num_data)

#シグモイド関数に入れる部分
z_base = x_1 + (1 - x_2)*10 - 40 + 5*e_z

#シグモイド関数を計算
z_prob = expit(0.1*z_base)

# テレビcmを見たかどうかの変数
Z = np.array([])


for i in range(num_data):
    Z_i = np.random.choice(2, size=1, p=[1 - z_prob[i], z_prob[i]])[0]
    Z = np.append(Z, Z_i)
    
#ノイズの生成
e_y = randn(num_data)

Y =  -x_1 + 30*x_2 + 10*Z + 80 + 10*e_y

In [4]:
df = pd.DataFrame({'年齢': x_1,
                   '性別': x_2,
                   'CMを見た': Z,
                   '購入量': Y,
                   })

In [5]:
df.head()

,年齢,性別,CMを見た,購入量
0,62,0,1.0,24.464285
1,34,0,0.0,45.693411
2,53,1,1.0,64.998281
3,68,1,1.0,47.186898
4,27,1,0.0,100.114260


In [9]:
#説明変数
X = df[["年齢", "性別"]]

#被説明変数（目的変数）
Z = df["CMを見た"]


#回帰の実態
reg = LogisticRegression().fit(X, Z)

#回帰した結果の係数を出力
print("係数beta : ", reg.coef_)
print("係数alpha : ", reg.intercept_)

係数beta :  [[ 0.10562765 -1.38263933]]
係数alpha :  [-3.37146523]


In [10]:
Z_pre = reg.predict_proba(X)
print(Z_pre[0:5])

[[0.04002323 0.95997677]
 [0.44525168 0.55474832]
 [0.30065918 0.69934082]
 [0.08101946 0.91898054]
 [0.87013558 0.12986442]]


In [11]:
ATE_i = Y / Z_pre[:, 1]*Z - Y / Z_pre[:, 0]*(1-Z)
ATE = 1 / len(Y) * ATE_i.sum()
print("推定したATE : ", ATE)

推定したATE :  8.847476810855458


In [18]:
#説明変数
X = df[["年齢", "性別", "CMを見た"]]

#被説明変数
y = df["購入量"]


#回帰の実態
reg2 = LinearRegression().fit(X, y)

#Z=0の場合
X_0 = X.copy()
X_0["CMを見た"] = 0
Y_0 = reg2.predict(X_0)


#Z=1の場合
X_1 = X.copy()
X_1["CMを見た"] = 1
y_1 = reg2.predict(X_1)

In [19]:
#傾向スコアを求める

#説明変数
X = df[["年齢", "性別"]]


#目的変数
Z = df["CMを見た"]


#回帰の実態
reg = LogisticRegression().fit(X, Z)


#傾向スコアを求める
Z_pre = reg.predict_proba(X)
print(Z_pre[0:5])

[[0.04002323 0.95997677]
 [0.44525168 0.55474832]
 [0.30065918 0.69934082]
 [0.08101946 0.91898054]
 [0.87013558 0.12986442]]


In [21]:
ATE_1_i = Y/Z_pre[:, 1] * Z + (1 - Z  / Z_pre[:, 1])*Y_1
ATE_0_i = Y / Z_pre[:, 0] * (1 - Z) + (1 - (1 - Z) / Z_pre[:, 0])*Y_0
ATE =   1 / len(Y) * (ATE_1_i - ATE_0_i).sum()
print("推定したATE:", ATE)

推定したATE: 9.75277505424846
